# Extract data from ObserveRTC

The output is a CSV file which will be appened to after each run.

A local file will keep track of when the script was run last and only query data for that given time period.

If there already exists a local data file, then it will load it and remove the latest duplicates. 

And append the newest entries. After this other data processing can happen by reading the CSV file.

In [120]:
!pip install pymongo
!pip install pandas

In [121]:
from pymongo import MongoClient
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time
import json
import pandas as pd

In [125]:
load_dotenv()
address = 'mongodb://{user}:{password}@{host}:{port}'.format(
    user= os.getenv('MONGO_USER'),
    password= os.getenv('MONGO_PASSWORD'),
    host= os.getenv('MONGO_HOST'),
    port= os.getenv('MONGO_PORT')
)
client = MongoClient(address)
database=client["observertc-reports"]

# the collection we want to query
reportsDatabase = database.reports

#read number from text file
last_id = 0
try:
    with open("last_id.txt", "r") as f:
        last_id = int(f.read())
except:
    pass

timeOfQuery = datetime.now()
print(last_id)

cursor = reportsDatabase.find({"type": "CLIENT_EXTENSION_DATA", 
                                "payload.extensionType" : { "$in" : [
                                    "OUT_BOUND_RTC", 
                                    "IN_BOUND_RTC", 
                                    "REMOTE_OUT_BOUND_RTC", 
                                    "REMOTE_IN_BOUND_RTC"]}, 
                                "payload.timestamp" : {"$gt": 1667994458489},
                                "payload.roomId" : "5553563"  })


dataSet = []
for record in cursor:
    data = {}
    #append timestamp to data
    data["timestamp"] = record["payload"]["timestamp"]
    data["callId"] = record["payload"]["callId"]
    data["roomId"] = record["payload"]["roomId"]
    data["clientId"] = record["payload"]["clientId"]
    data["userId"] = record["payload"]["userId"]
    data["sampleSeq"] = record["payload"]["sampleSeq"]

    a = json.loads(record["payload"]["payload"])

    # https://stackoverflow.com/questions/38987/how-do-i-merge-two-dictionaries-in-a-single-expression
    data = {**data, **a["stats"]}

    dataSet.append(data)

#outBound = sorted(outBound, key=lambda k: k['timestamp'])

newData = pd.DataFrame(dataSet)
newData["callId"]=newData["callId"].astype(str)
newData["roomId"]=newData["roomId"].astype(str)
newData["clientId"]=newData["clientId"].astype(str)
newData["userId"]=newData["userId"].astype(str)

#df.to_csv("data.csv", index=False)


# Append data to existing csv but skip duplicates
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
oldData = pd.read_csv("data.csv")

oldData["callId"]=oldData["callId"].astype(str)
oldData["roomId"]=oldData["roomId"].astype(str)
oldData["clientId"]=oldData["clientId"].astype(str)
oldData["userId"]=oldData["userId"].astype(str)

print("size of old data: ", oldData.shape)
print("size of new data: ", newData.shape)

on = ['timestamp', 'callId', 'roomId', 'clientId', 'userId', 'type', 'sampleSeq', 'kind']
diff =pd.merge(oldData,newData[on], indicator=True, how='right', on=on).query('_merge=="right_only"').drop('_merge', axis=1)

print("oldData: ", oldData.shape, " newData: ", newData.shape, "diff: ", diff.shape)

# append diff to data.csv
diff.to_csv("data.csv", mode='a', header=False, index=False)

0
size of old data:  (7134, 63)
size of new data:  (7134, 63)
oldData:  (7134, 63)  newData:  (7134, 63) diff:  (0, 63)
